# Vehicle Classification Report on original dataset

In [2]:
import os

from sklearn.metrics import accuracy_score, classification_report

from utils import utils
from models import resnet_50

In this first section are shown some results obtained training our model with the original data set, without removing background.

Different configurations were tested locally in a small section with quite large learning rates (~0.01) of the dataset for fast convergencies to get an overall idea on how each of the parameters influence in the final accuracy.

From those experiments better results were obtained with low values of zoom factors, only horizontal mode for the flip, low values of kernel_regularizer and high dropout values.

Trying this in the complete dataset gave a validation accuracy around 0.24 without data_augmentation and 0.32 with it

In order to improve these results, more options to the configuration files were added.
More dense layers with ReLuu activation were added beetween the resenet model and the output layer as described [here](https://hamdi-ghorbel78.medium.com/a-guide-to-transfer-learning-with-keras-using-resnet50-453934a7b7dc) and also as it is known that as deeper a layer in a neural network is, it gets more specialized capturing features from the training set, and as this model was pre trained with another dataset, a function to unfreeze last layers of the renet model was added keeping the batch normalization layer frozen as suggested [here](https://stackoverflow.com/questions/44609533/can-we-use-batch-normalization-with-transfer-learning-for-an-instance-with-diffe.)

## TEST #1

In this first test, 2 aditional dense layer were added and 3 (n_unfreeze_layers: 8) layer were unfrozen from the `resnet_50 model`

A validation accuracy of 0.53 was obtained

In [3]:
CONFIG_YML = "../experiments/exp_1/config.yml"
WEIGHTS = "../experiments/exp_1/model.51-2.6121.h5"
config = utils.load_config(CONFIG_YML)
config

{'seed': 123,
 'data': {'directory': '/home/app/src/data/car_ims_v1/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': 'imagenet',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'dropout_rate': 0.5,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_rotation': {'factor': 0.3},
   'random_zoom': {'height_factor': 0.2, 'width_factor': 0.2}},
  'regularizers': {'kernel_regularizer': {'L2': {'l2': 0.001}},
   'activity_regularizer': {'L2': {'l2': 0.001}}},
  'output_regularizer': {'kernel_regularizer': {'L2': {'l2': 0.001}}},
  'n_dense_layers': 2,
  'n_unfreeze_layers': 8,
  'trainable': True},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.0001}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 100,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/experiments/exp_1/model.{epoch:02d}-{val_loss:.4f

In [4]:
# Infer the class names and also load the corresponding testing dataset.

MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

### Loading the model


In [5]:
cnn_model = resnet_50.create_model(weights=WEIGHTS, trainable=False)
print(cnn_model.summary())

2022-11-21 15:36:42.803479: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-21 15:36:42.803525: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-21 15:36:42.803548: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jotace): /proc/driver/nvidia/version does not exist
2022-11-21 15:36:42.803916: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                             

### Get predictions from testing dataset

In [6]:

#     Get model predictions and the corresponding true labels
#     so we can measure the accuracy

predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


In [7]:
print(classification_report(y_true=labels, y_pred=predictions))

                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.80      0.91      0.85        44
                             Acura Integra Type R 2001       0.74      0.52      0.61        44
                                   Acura RL Sedan 2012       0.25      0.09      0.14        32
                                   Acura TL Sedan 2012       0.48      0.60      0.54        43
                                  Acura TL Type-S 2008       0.40      0.33      0.36        42
                                  Acura TSX Sedan 2012       0.50      0.23      0.31        40
                              Acura ZDX Hatchback 2012       0.45      0.33      0.38        39
              Aston Martin V8 Vantage Convertible 2012       0.28      0.51      0.37        45
                    Aston Martin V8 Vantage Coupe 2012       0.38      0.22      0.28        41
                  Aston Martin Virage C

In [8]:
acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.5248!


## Test 2

For this test a fine-tuning was performed starting from the previous weights. Also a callback to reduce the learning rate in case the evaluation accuracy stops increasing was added.

A validation accuracy of 0.7654 was obtained

In [21]:
CONFIG_YML_RETRAIN = "../experiments/exp_2/config.yml"
WEIGHTS_RETRAIN = "../experiments/exp_1/model.51-2.612.h5"
config_retrain = utils.load_config(CONFIG_YML_RETRAIN)
config_retrain

{'seed': 123,
 'data': {'directory': '/home/app/src/data/car_ims_v1/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': '/home/app/src/experiments/exp_1/model.51-2.6121.h5',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'dropout_rate': 0.5,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_rotation': {'factor': 0.3},
   'random_zoom': {'height_factor': 0.2, 'width_factor': 0.2}},
  'regularizers': {'kernel_regularizer': {'L2': {'l2': 0.001}},
   'activity_regularizer': {'L2': {'l2': 0.001}}},
  'output_regularizer': {'kernel_regularizer': {'L2': {'l2': 0.001}}},
  'n_dense_layers': 2,
  'n_unfreeze_layers': 8,
  'trainable': True},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.0001}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 100,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/experim

In [10]:
cnn_model_retrain = resnet_50.create_model(weights=WEIGHTS_RETRAIN, trainable=False)
print(cnn_model_retrain.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                             

In [11]:

#     Get model predictions and the corresponding true labels
#     so we can measure the accuracy

predictions_retrain, labels_retrain = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model_retrain, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions_retrain) != len(labels_retrain):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions_retrain[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels_retrain[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


In [12]:
print(classification_report(y_true=labels_retrain, y_pred=predictions_retrain))

                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.91      0.95      0.93        44
                             Acura Integra Type R 2001       0.84      0.82      0.83        44
                                   Acura RL Sedan 2012       0.60      0.28      0.38        32
                                   Acura TL Sedan 2012       0.64      0.88      0.75        43
                                  Acura TL Type-S 2008       0.58      0.81      0.67        42
                                  Acura TSX Sedan 2012       0.74      0.65      0.69        40
                              Acura ZDX Hatchback 2012       0.79      0.59      0.68        39
              Aston Martin V8 Vantage Convertible 2012       0.50      0.53      0.52        45
                    Aston Martin V8 Vantage Coupe 2012       0.58      0.46      0.51        41
                  Aston Martin Virage C

In [13]:
acc = accuracy_score(y_true=labels_retrain, y_pred=predictions_retrain)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.7531!
